In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../metnet')
sys.path.append('../common')

In [3]:
from metnet import MetNet
import torch
import torch.nn.functional as F

import loss_utils

In [4]:
FORECAST = 10
GROUP = 40
OUTPUTS = round(1023 / GROUP) + 1
model = MetNet(
        hidden_dim=32,
        forecast_steps=FORECAST, # should be 24 timesteps out
        input_channels=1, # 12 timeteps in
        output_channels=OUTPUTS, # 1 data channel in
        sat_channels=1, # 1 data channel in
        input_size=32, # =128/4, where 128 is the image dimensions
)

In [5]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [6]:
model = model.to(DEVICE)

In [7]:
sum(p.numel() for p in model.parameters())

1823163

In [8]:
torch.manual_seed(7)
BATCH_SIZE = 1
x = torch.randn((BATCH_SIZE, 12, 1, 128, 128)).to(DEVICE)
# y = torch.randn((1, FORECAST, 1, 64, 64)).to(DEVICE)
y = torch.round(torch.randint(0, 1024, (1, FORECAST, 64, 64)) / GROUP).long().to(DEVICE)
# criterion = loss_utils.MS_SSIMLoss(channels=FORECAST)
criterion = torch.nn.CrossEntropyLoss()

out = model(x)

/home/sigaida/miniconda3/envs/climatehack/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/sigaida/miniconda3/envs/climatehack/lib/python3.9/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [9]:
out.shape

torch.Size([1, 10, 27, 64, 64])

In [10]:
y.shape

torch.Size([1, 10, 64, 64])

In [11]:
t1 = out.reshape(-1, OUTPUTS, 64, 64)
t2 = y.reshape(-1, 64, 64)

In [12]:
t1.shape, t2.shape

(torch.Size([10, 27, 64, 64]), torch.Size([10, 64, 64]))

In [13]:
t1[0,0,:8,:8]

tensor([[-0.1246, -0.1359, -0.1059, -0.1241, -0.0854, -0.1297, -0.0963, -0.1012],
        [-0.1253, -0.1348, -0.1007, -0.1237, -0.0817, -0.1265, -0.0936, -0.1057],
        [-0.1242, -0.1350, -0.1055, -0.1265, -0.0869, -0.1309, -0.0975, -0.1022],
        [-0.1364, -0.1484, -0.1125, -0.1299, -0.0944, -0.1393, -0.1126, -0.1096],
        [-0.1095, -0.1217, -0.0876, -0.1094, -0.0684, -0.1150, -0.0813, -0.0851],
        [-0.1168, -0.1212, -0.0928, -0.1071, -0.0731, -0.1155, -0.0833, -0.0956],
        [-0.1239, -0.1318, -0.1007, -0.1271, -0.0872, -0.1297, -0.0946, -0.1101],
        [-0.1082, -0.1223, -0.0911, -0.1075, -0.0671, -0.1182, -0.0800, -0.0895]],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [14]:
loss = criterion(t1[:1], t2[:1])

In [15]:
loss

tensor(3.2970, device='cuda:0', grad_fn=<NllLoss2DBackward0>)

In [16]:
loss.backward()